In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
titanic.groupby("sex")["survived"].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

Veri setini cinsiyet değişkenini referans alarak survive değişkeninin değerlerinin(hayatta kalma değerleri) ortalamsına ulaştık

Bu yapılan işlem ilkel bir pivot tablo oluşturma işlemidir

In [5]:
titanic.groupby("sex")[["survived"]].mean()

,survived
sex,
female,0.742038
male,0.188908


Değişkenin formunu değiştirerek([ ] içine aldık) df formatında görüntüleyebiliriz

Şimdi benzer işlemlerle ve çoklu parametrelerle bir pivot tablo oluşturalım

In [8]:
titanic.groupby(["sex","class"])[["survived"]].aggregate("mean").unstack()

survived                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

groupby metoduna gönderilen parametre sayısını artırmak yani gruplamayı birden fazla değişken için yapmak istiyorsak bu değişkenler parametreye dizi şeklinde gönderilir

Çoklu gruplama işleminde **unstack** metodu kullanılırsa: ilk gönderilen değişken x ekseni(satır)nde , diğer değişken ise y ekseninde yer alır ve bunların kesişimleri şeklinde veriler döndürülür

        unstack metodunu kullanmadan ayni pivot tabloyu tekrar oluşturalım

In [10]:
titanic.groupby(["sex","class"])[["survived"]].aggregate("mean")

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

unstack metodunu kullanmadığımızda gruplama yaptığımız her iki değişkende x ekseninde yer alır

İlk değişken ana referans alınan ana değişken gibi düşünülürse sonraki değişken ise bu değişken altında bulunan diğer bir sınıf değişkeni olarak düşünülebilir

Ve gruplama yapılan bu iki değişkenin istenen değişkendeki kesişimleri aggregate fonksiyonuna tabi tutularak pivot tablo oluşturulur

Ortalama ile oluşturduğumuz bu pivot tabloyu **pivot_table** metoduyla daha kolay şekilde oluşturabiliriz

In [12]:
titanic.pivot_table("survived",index="sex",columns="class")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


* ilk parametreye ortalamanın alıncağı değişken (odak değişken) girilir
* index parametresine referans alınacak (yukarıdaki işleme göre gruplanacak) değişken girilir
* columns parametresine de gruplamaya dahil edeceğimiz ikinci değişken girilir
* metot yine index ve columns parametredeki değerlerin kesişimine ulaşarak ilk parametresindeki değişkenin değerlerinin ortalamasını alır
* unstack() metodu kullanılmış gibi pivot tablo oluşturur

In [13]:
titanic.age.head()

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: age, dtype: float64

In [14]:
age = pd.cut(titanic["age"],[0,18,90])
age.head(10)

0    (18.0, 90.0]
1    (18.0, 90.0]
2    (18.0, 90.0]
3    (18.0, 90.0]
4    (18.0, 90.0]
5             NaN
6    (18.0, 90.0]
7     (0.0, 18.0]
8    (18.0, 90.0]
9     (0.0, 18.0]
Name: age, dtype: category
Categories (2, interval[int64]): [(0, 18] < (18, 90]]

df içinde sürekli değişken olan age değişkenini kategorik bir değişken olarak değiştirdik

* cut metoduyla bölmek istediğimiz değişkeni kullanacağımız df üzerinden ulaşarak gönderdik
* ikinci parametre olarak dizi şeklinde bir veri yolladık
* cut metodu ikinci parametreyi referans alarak age değişkenlerini gruplayarak(sınıf haline getirerek) kategorik değişken haline getirecektir
* Bu kategorik değişkenin sınıflarını inceleyelim:
* ilk sınıf: 0-18 yaş arasındakiler,
* ikinci sınıf: 18-90 yaş arasındakiler

In [15]:
titanic.pivot_table("survived",["sex",age],"class")

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 90]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 90]  0.375000  0.071429  0.133663

* ilk parametreye yine odak değişken girildi
* ikinci parametre olarak gönderdiğimiz dizi ise index parametresinin karşılığı olmuştur
* class değişkenini alan parametre ise columns parametresi olmuştur

Bu sayede dışarıda, ana tablo üzerinden oluşturduğumuz yeni bir değişkeni pivot tablomuza bir boyut olarak ekledik

**age değişkeni dışarıdan alındığı için string şeklinde yazılmaz**